## This notebook Contains:
- Taking scraped input(HTML formatted code)
- Cleaning, Data Preprocessing and Feature Engineering on the data set
- Importing the Cleaned CSV File

In [7]:
# imoporting libraries
import pandas as pd
import os
from bs4 import BeautifulSoup
import re

In [20]:
# Reading the list of files inside the HTML_FILES folder
allfileslist = os.listdir("../../Data/HTML_FILES/")
# Concatenating all the files in the HTML_FILES folder
combined_csv = pd.concat( [ pd.read_csv("../../Data/HTML_FILES/"+f) for f in allfileslist ] )

In [21]:
# count of accept and reject
combined_csv.status.value_counts()

reject     5938
accept     5300
acccept     194
Name: status, dtype: int64

In [22]:
combined_csv.loc[combined_csv['status']=='acccept',"status"]="accept"

In [23]:
combined_csv.status.value_counts()

reject    5938
accept    5494
Name: status, dtype: int64

In [24]:
#filtering out empty records
combined_csv=combined_csv.loc[~(combined_csv['links']=="[]"),:]

In [25]:
#removing empty records
combined_csv.drop(columns='Unnamed: 0',inplace=True)
combined_csv.reset_index(drop=True,inplace=True)

In [26]:
# Changing university name in proper naming convention
combined_csv.loc[combined_csv.loc[:,'university_name']=="illinois_institute_of_technology_accept","university_name"]="illinois_institute_of_technology"

In [27]:
# Changing university name in proper naming convention
combined_csv.loc[combined_csv.loc[:,'university_name']=="university of california, irvine","university_name"]="university_of_california_irvine"

In [28]:
# Changing university name in proper naming convention
combined_csv.loc[combined_csv.loc[:,'university_name']=="clemson_university_accept","university_name"]="clemson_university"
combined_csv.loc[combined_csv.loc[:,'university_name']=="clemson_university_reject","university_name"]="clemson_university"

In [29]:
# Changing university name in proper naming convention
combined_csv.loc[combined_csv.loc[:,'university_name']=="university_of_texas_dallas_accept","university_name"]="university_of_texas_dallas"
combined_csv.loc[combined_csv.loc[:,'university_name']=="university_of_texas_dallas_reject","university_name"]="university_of_texas_dallas"

In [30]:
# Accept and Reject for every university with percentage of accept and reject
combined_csv.groupby(by=["university_name"])['status'].value_counts(normalize=True)

university_name                              status
carnegie mellon university                   reject    0.741379
                                             accept    0.258621
clemson_university                           accept    0.671875
                                             reject    0.328125
george mason university                      accept    0.639576
                                             reject    0.360424
georgia institiute of technology             accept    0.687943
                                             reject    0.312057
illinois_institute_of_technology             accept    0.622601
                                             reject    0.377399
indiana_university_bloomington               accept    0.802740
                                             reject    0.197260
kansas state university                      accept    0.597561
                                             reject    0.402439
michigan_technological_university            accept 

In [31]:
# shape of the datset
combined_csv.shape

(9889, 3)

In [32]:
#unwrapping stored html pages and extracting features from html tags
html_pages = combined_csv.links.tolist()
temp = []

In [33]:
# Function to unwrap the html
for i in html_pages:
    soup = BeautifulSoup(i)
    a = soup.find_all('div', class_ = 'col-sm-4 col-xs-4')
    temp_inside = []
    for x in a:
        k =(x.h4.text)
        t=[j for j in k.strip().split("\n") if len(j) is not 0]
        temp_inside.append(t)
    temp.append(temp_inside)

In [36]:
temp[0:1]

[[['GRE 316 ', '\r', 'Quant: 164\r', 'Verbal: 152'],
  ['TOEFL106'],
  ['UG Score\r', '78%'],
  ['Work Exp.9 months'],
  ['Tech PapersInternational'],
  ['Dream Univ.USC']]]

In [37]:
# getting all the profile data in nested list and extracting it
all=[]
for each in temp:
    list = []
    for i in each:
        for j in i:
            list.append(j)
    all.append(list)

In [38]:
#verifing if we have unpacked all html pages collected correctly
len(all)

9889

In [39]:
all[0]

['GRE 316 ',
 '\r',
 'Quant: 164\r',
 'Verbal: 152',
 'TOEFL106',
 'UG Score\r',
 '78%',
 'Work Exp.9 months',
 'Tech PapersInternational',
 'Dream Univ.USC']

In [40]:
#we will make a new dataframe with extracted information from html pages and it's corresponding university name and status
university_list=combined_csv.university_name.tolist()
status_list=combined_csv.status.tolist()

In [41]:
combined_df = pd.DataFrame(all)
combined_df['university_name']=university_list
combined_df['status']=status_list

In [42]:
#naming our features
list_columns = ['gre_score','droping', 'gre_score_quant','gre_score_verbal','test_score_toefl','droping_1', 'undergraduation_score','work_ex', 'papers_published','droping_3','university_name','status']
combined_df.columns = list_columns
combined_df.drop(columns = ['droping','droping_1','droping_3'], inplace=True)

In [43]:
# Null in columns
combined_df.isna().sum()

gre_score                  0
gre_score_quant            0
gre_score_verbal           0
test_score_toefl           0
undergraduation_score      0
work_ex                  242
papers_published         242
university_name            0
status                     0
dtype: int64

In [44]:
#filling work experience and work_ex with zero, considering when there are no values given
combined_df=combined_df.fillna(0)

In [45]:
combined_df.head()

,gre_score,gre_score_quant,gre_score_verbal,test_score_toefl,undergraduation_score,work_ex,papers_published,university_name,status
0,GRE 316,Quant: 164\r,Verbal: 152,TOEFL106,78%,Work Exp.9 months,Tech PapersInternational,northeastern university,accept
1,GRE 314,Quant: 163\r,Verbal: 151,IELTS8,8.82CGPA,Work Exp.7 months,Tech PapersInternational,northeastern university,accept
2,GRE 307,Quant: 160\r,Verbal: 147,TOEFL88,80%,Work Exp.36 months,Tech PapersNone,northeastern university,accept
3,GRE 330,Quant: 169\r,Verbal: 161,TOEFL113,68.05%,Work Exp.48 months,Tech PapersInternational,northeastern university,accept
4,GRE 316,Quant: 163\r,Verbal: 153,TOEFL99,74.2%,Work Exp.30 months,Tech PapersNone,northeastern university,accept


<b>Data Pre processing and Feature Engineering</b>

- Removing Null values from columns
- Removing noise data, Unformatted Text and Inconsistent Data 
- Conversion of % and 10 pinter score in CGPA to 4 pointer
- Toefl and IELTS score to the same scale according to the information available on ETS Official website (https://www.ets.org/toefl/institutions/scores/compare/)
- Including Ranking of University as a column
- Changed paper Published containing column values as NoneInternational/National/Local

In [46]:
# Function for removing special charaters 
def replace_special_chars(i):
    #a = re.sub('[^A-Za-z]+',' ',str(i))
    a=re.findall(r'\d+', str(i))
    #a = a.lower()
    return ''.join(a)

In [47]:
# calling this function for various columns

In [48]:
combined_df['gre_score']=combined_df.gre_score.apply(replace_special_chars)
combined_df['gre_score_quant']=combined_df['gre_score_quant'].apply(replace_special_chars)

In [49]:
combined_df['test_score_toefl'] = combined_df['test_score_toefl'].apply(replace_special_chars)
combined_df['gre_score_verbal'] = combined_df['gre_score_verbal'].apply(replace_special_chars)
combined_df['work_ex'] = combined_df['work_ex'].apply(replace_special_chars)

In [50]:

combined_df["undergraduation_score"] = [x.replace('CGPA','') for x in combined_df["undergraduation_score"]]
combined_df["undergraduation_score"] = [x.replace('%','') for x in combined_df["undergraduation_score"]]
combined_df["papers_published"] = [str(x).replace('Tech Papers','') for x in combined_df["papers_published"]]

In [51]:
# data type for multiple columns
combined_df.dtypes

gre_score                object
gre_score_quant          object
gre_score_verbal         object
test_score_toefl         object
undergraduation_score    object
work_ex                  object
papers_published         object
university_name          object
status                   object
dtype: object

In [52]:
combined_df.loc[combined_df['work_ex']=='','work_ex']=0

In [53]:
values=[]
for each in combined_df.undergraduation_score.unique():
    try:
        float(each)
    except:
        values.append(each)

In [54]:
for each in values:
    combined_df=combined_df[combined_df.undergraduation_score!=each]

In [55]:
combined_df[['gre_score','gre_score_quant','gre_score_verbal','test_score_toefl','undergraduation_score','work_ex']]=combined_df[['gre_score','gre_score_quant','gre_score_verbal','test_score_toefl','undergraduation_score','work_ex']].apply(pd.to_numeric)

In [56]:
combined_df=combined_df.loc[~(combined_df.test_score_toefl.isna()),:]

In [57]:
combined_df.isna().sum()

gre_score                0
gre_score_quant          0
gre_score_verbal         0
test_score_toefl         0
undergraduation_score    0
work_ex                  0
papers_published         0
university_name          0
status                   0
dtype: int64

In [58]:
combined_df.reset_index(drop=True,inplace=True)

In [59]:
# function to scale the cgpa on the scale of 4
update_cgpa_score_scale_4 = []
for score in combined_df.undergraduation_score.tolist():
   s = 0
   try:
       score = float(score)
   except:
       score= 0
   if score > 10:
       s = ((score)/20) - 1
       s = round(s,2)
       update_cgpa_score_scale_4.append(s)
   else:
       s = ((score)/10)*4
       s = round(s,2)
       update_cgpa_score_scale_4.append(s)

In [60]:
combined_df['undergraduation_score']=update_cgpa_score_scale_4

In [61]:
combined_df.loc[combined_df['test_score_toefl']<9,'test_score_toefl']=pd.cut(combined_df.loc[combined_df['test_score_toefl']<9,'test_score_toefl'], bins=[-1,0.5,4,4.5,5,5.5,6,6.5,7,7.5,8,8.5,9], labels=[0,31,34,45,59,78,93,101,109,114,117,120])

In [62]:
combined_df.loc[combined_df['test_score_toefl']<9,'test_score_toefl'].value_counts()

0    353
Name: test_score_toefl, dtype: int64

##### working on the paper published column to assign values: International as 3, National as 2, Local as 1 and None as 0

In [63]:
combined_df.papers_published.unique()

array(['International', 'None', 'NA', 'Local', 'National', ''],
      dtype=object)

In [64]:
#df_all_neu["papers_published"] = [x.replace('','0') for x in df_all_neu["papers_published"]]
combined_df["papers_published"] = [x.replace('None','0') for x in combined_df["papers_published"]]
combined_df["papers_published"] = [x.replace('NA','0') for x in combined_df["papers_published"]]

In [65]:
combined_df.papers_published.value_counts()

0                6753
International    1872
National          458
Local             260
                    9
Name: papers_published, dtype: int64

In [66]:

combined_df.loc[combined_df['papers_published'] == 'Local', 'papers_published'] = '1'
combined_df.loc[combined_df['papers_published'] == 'International', 'papers_published'] = '3'
combined_df.loc[combined_df['papers_published'] == 'National', 'papers_published'] = '2'

In [67]:
list_ppr_pub = combined_df.papers_published.tolist()

In [68]:
new_list_ppr_pub = []
for i in list_ppr_pub:
    if i == '':
        new_list_ppr_pub.append('0')
    else:
        new_list_ppr_pub.append(i)

In [69]:
combined_df['papers_published'] = new_list_ppr_pub

In [70]:
combined_df['papers_published'] = combined_df['papers_published'].astype(int)

In [71]:
combined_df.describe()


,gre_score,gre_score_quant,gre_score_verbal,undergraduation_score,work_ex,papers_published
count,9352.000000,9352.000000,9352.000000,9352.000000,9352.000000,9352.000000
mean,314.110351,162.800791,151.309559,3.029532,16.853935,0.726262
std,9.301243,5.229764,6.040880,0.488298,16.127780,1.223903
min,0.000000,0.000000,0.000000,1.060000,0.000000,0.000000
25%,308.000000,160.000000,148.000000,2.720000,2.000000,0.000000
50%,314.000000,163.000000,151.000000,3.100000,15.000000,0.000000
75%,320.000000,167.000000,155.000000,3.400000,27.000000,1.000000
max,340.000000,170.000000,170.000000,4.000000,153.000000,3.000000


#### checking and removing incorrect record
Gre quant/verbal >170 and <130

In [72]:
combined_df.loc[(combined_df['gre_score_quant'] <130) | (combined_df['gre_score_verbal'] < 130)  | (combined_df['gre_score'] < 260),:]

,gre_score,gre_score_quant,gre_score_verbal,test_score_toefl,undergraduation_score,work_ex,papers_published,university_name,status
6230,204,62,142,103,2.50,0,0,syracuse_university,reject
7012,0,0,0,90,2.55,0,0,university of colorado boulder,reject


In [73]:
combined_df = combined_df.loc[~((combined_df['gre_score_quant'] <130) | (combined_df['gre_score_verbal'] < 130)  | (combined_df['gre_score'] < 260)),:]

In [75]:
# No null columns remaining
combined_df.isna().sum()

gre_score                0
gre_score_quant          0
gre_score_verbal         0
test_score_toefl         0
undergraduation_score    0
work_ex                  0
papers_published         0
university_name          0
status                   0
dtype: int64

In [76]:
def replace_special_chars_university_name(i):
    a = re.sub('[^A-Za-z]+',' ',str(i))
    #a=re.findall(r'\d+', str(i))
    a = a.lower()
    
    return '_'.join(a.split(' '))

In [77]:
#replacing special characters and spaces in university name
combined_df.loc[:,"university_name"]=combined_df.university_name.apply(replace_special_chars_university_name)

In [78]:
required_colleges=combined_df.university_name.unique().tolist()
len(required_colleges)

29

In [79]:
required_colleges=['northeastern_university','illinois_institute_of_technology','michigan_technological_university','rochester_institute_of_technology','university_of_southern_california','north_carolina_state_university_raleigh','university_of_texas_arlington','university_of_texas_dallas','syracuse_university','clemson_university','new_york_university','indiana_university_bloomington','rutgers_university_new_brunswick', "---",'university_of_florida','carnegie_mellon_university','georgia_institiute_of_technology','university_of_colorado_boulder','university_of_north_carolina_at_charlotte','university_of_iowa','university_of_connecticut','worcester_polytechnic_institute','---','kansas_state_university','university_of_cincinnati','university_of_maryland_college_park','university_of_california_irvine','texas_a_m_university_college_station','state_university_of_new_york_at_stony_brook','george_mason_university','university_of_texas_austin']

In [80]:
# Assigining universities with their respective rankings in CS
required_colleges_ranking = [15,97,117,66,19,49,64,52,118,89,22,48,25,150,62,1,9,58,30, 71, 70,79, 76, 115, 130, 10, 23, 31, 35, 59,16]
dictionary_req_college = dict(zip(required_colleges, required_colleges_ranking))
dictionary_req_college
combined_df['ranking'] = combined_df['university_name']
combined_df['ranking'].replace(dictionary_req_college,inplace=True)

In [81]:
# no null values remaining
combined_df.isna().sum()

gre_score                0
gre_score_quant          0
gre_score_verbal         0
test_score_toefl         0
undergraduation_score    0
work_ex                  0
papers_published         0
university_name          0
status                   0
ranking                  0
dtype: int64

In [83]:
# cleaned datset 
combined_df.head()

,gre_score,gre_score_quant,gre_score_verbal,test_score_toefl,undergraduation_score,work_ex,papers_published,university_name,status,ranking
0,316,164,152,106,2.90,9,3,northeastern_university,accept,15
1,314,163,151,114,3.53,7,3,northeastern_university,accept,15
2,307,160,147,88,3.00,36,0,northeastern_university,accept,15
3,330,169,161,113,2.40,48,3,northeastern_university,accept,15
4,316,163,153,99,2.71,30,0,northeastern_university,accept,15


In [84]:
# describing the dataset
combined_df.describe()

,gre_score,gre_score_quant,gre_score_verbal,undergraduation_score,work_ex,papers_published,ranking
count,9350.000000,9350.000000,9350.000000,9350.000000,9350.000000,9350.000000,9350.000000
mean,314.155722,162.828984,151.326738,3.029640,16.857540,0.726417,47.039251
std,8.641709,4.840835,5.834517,0.488294,16.127621,1.223988,33.960770
min,260.000000,130.000000,130.000000,1.060000,0.000000,0.000000,1.000000
25%,308.000000,160.000000,148.000000,2.720000,2.000000,0.000000,16.000000
50%,314.000000,163.000000,151.000000,3.100000,15.000000,0.000000,35.000000
75%,320.000000,167.000000,155.000000,3.400000,27.000000,1.000000,64.000000
max,340.000000,170.000000,170.000000,4.000000,153.000000,3.000000,130.000000


In [100]:
# transferring CSV file
combined_df.reset_index(drop =True).to_csv('../../Data/clean_profile_data_all.csv',index=False)